## LSTM!!!

In [1]:
#from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 85.0% of memory, cuDNN 5110)


In [2]:
text = open('datasetV2twplustag').read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

('corpus length:', 896291)
('total chars:', 40)


In [57]:
with open('datasetV2twplustag') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
f = [x.strip().lower() for x in content] 

end = []
vri = []
for i in range(len(content)):
    end.append(content[i].index('['))
    vri.append(content[i].index(']')-content[i].index('[')+1)
    content[i] = content[i][:-1].lower()
    
sentences = content
#sentencesRaw = []
#next_chars = []
#for i in range(len(content)):
#    sentencesRaw.append(content[i][:end[1]])
#    next_chars.append('[')

In [70]:
maxlen = len(max(content, key=len))

X = np.zeros((sum(vri), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((sum(vri), len(chars)), dtype=np.bool)


for i, sentence in enumerate(sentences):
    for j in range(vri[i]):
        for t, char in enumerate(sentence[:-j]):
            X[i, t+(maxlen-len(sentence)+j), char_indices[char]] = 1
        y[i, char_indices[sentence[-j]]] = 1
        
print X.shape

(128049, 144, 40)


In [71]:
print('Build model...')
model = Sequential()
#model.add(LSTM(4, input_shape=(1, 1)))
model.add(LSTM(128, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float32')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Build model...


In [73]:
for iteration in range(10):
    print()
    print('-' * 80)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=256,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    

()
--------------------------------------------------------------------------------
('Iteration', 0)
Epoch 1/1
128049/128049 [==============================] - 167s - loss: 1.9321   
()
--------------------------------------------------------------------------------
('Iteration', 1)
Epoch 1/1
128049/128049 [==============================] - 167s - loss: 1.9296   
()
--------------------------------------------------------------------------------
('Iteration', 2)
Epoch 1/1
128049/128049 [==============================] - 167s - loss: 1.9277   
()
--------------------------------------------------------------------------------
('Iteration', 3)
Epoch 1/1
128049/128049 [==============================] - 167s - loss: 1.9273   
()
--------------------------------------------------------------------------------
('Iteration', 4)
Epoch 1/1
 89856/128049 [====================>.........] - ETA: 49s - loss: 1.9264

KeyboardInterrupt: 

In [69]:
for diversity in [0.2, 0.5, 1.0, 1.5]:
    print('diversity:', diversity)

    generated = ''
    sentence = content[1][:end[1]+1]#text[start_index: start_index + maxlen]
    generated += sentence
    generated += '['
    print 'Generating with seed:\n"' + sentence[:] +'"'
    print 'real hashtag: ' + content[1][end[1]:]
    #print 'generated hashtag: ['
    #sys.stdout.write(generated+'\n[')
    sys.stdout.write('generated hashtag: [')
    
    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
        
        if (next_char==']'):
            break
        
    print '\n', '\n'

6
100
('diversity:', 0.2)
Generating with seed:
"no matter what happens the only way i know is to work hard every day thank you all for your support ["
real hashtag: [mufc]
generated hashtag: []


('diversity:', 0.5)
Generating with seed:
"no matter what happens the only way i know is to work hard every day thank you all for your support ["
real hashtag: [mufc]
generated hashtag: [egeaieehreeri]


('diversity:', 1.0)
Generating with seed:
"no matter what happens the only way i know is to work hard every day thank you all for your support ["
real hashtag: [mufc]
generated hashtag: [rcsf2iktosopnseskfc]


('diversity:', 1.5)
Generating with seed:
"no matter what happens the only way i know is to work hard every day thank you all for your support ["
real hashtag: [mufc]
generated hashtag: []




In [ ]:
print len(vri)
print sum(vri)

In [ ]:
print max(content, key=len)

In [ ]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
